# Base.convert to String ambiguous issue

In [ ]:
using Dates

In [ ]:
import Base.convert

In [ ]:
Base.convert(String, date::DateTime) = "$date"

In [ ]:
convert

In [ ]:
convert

In [ ]:
struct MyStruct
    a::Integer
end

In [ ]:
MyStruct(1)

In [ ]:
# CAUTION this is WRONG, this is causing IJulia to restart
Base.convert(String, x::MyStruct) = "$x.a"

In [ ]:
# I should be using ::Type{String} instead!
Base.convert(::Type{String}, x::MyStruct) = "$(x.a)"

In [ ]:
Base.convert(String, MyStruct(1))

In [ ]:
String

In [ ]:
::Type{String}

In [ ]:
Base.convert

# GPU incompatibility

In [ ]:
using CUDA

In [ ]:
logistic(x) = 1 / (1 + exp(-x))

In [ ]:
A = cu(rand(5, 10))

In [ ]:
logistic.(A)

# Clean up saved models

In [ ]:
include("exp.jl")

In [ ]:
for dir in readdir_nohidden("saved_models")
    keep_most_recent!(joinpath("saved_models", dir))
end

# CUDA memory limit

In [ ]:
# setup GPU memory limit

# For 1070
g=5.0
# For 2080 Ti
# g=9.0

ENV["JULIA_CUDA_MEMORY_LIMIT"] = convert(Int, round(g * 1024 * 1024 * 1024))

# Model size

In [ ]:
import Printf

In [ ]:
function test_size()
    # model size
    @info "FC model"
    for d in [7, 10,15,20,25,30]
        Printf.@printf "%.2f\n" param_count(fc_model_fn(d)) / 1e6
    end
    @info "FC deep model"
    for d in [7, 10,15,20,25,30]
        Printf.@printf "%.2f\n" param_count(deep_fc_model_fn(d)) / 1e6
    end
    # EQ models is independent of input size
    @info "EQ model"
    Printf.@printf "%.2f\n" param_count(eq_model_fn(10)) / 1e6
    Printf.@printf "%.2f\n" param_count(deep_eq_model_fn(10)) / 1e6
end

# Train ensembly with different K

- because K is probably the most important hyper-parameter for real data
- [X] the data could be loaded more effectively, e.g. inside one dataset `ds`. Then the data can be completely mixed.

experiments:
- [X] try ER/SF graphs
- [ ] try COR mat
- [-] try normalized COV (medCOV and maxCOV)
- [ ] try COR + VAR as input
- [X] try similar k (1,2,3,4]: this works
- [X] try larger batch UPDATE but 100 should be large enough

In [ ]:
include("exp.jl")

In [ ]:
# first try to use multiple dses for training
specs = []
for k in [1,5,10,20]
    push!(specs, DataSpec(d=11, k=k, gtype=:SF, noise=:Gaussian, mat=:maxCOV))
end
specs = Array{DataSpec}(specs)

In [ ]:
# train
expID = exp_train(specs, deep_eq_model_fn, prefix="ensK", train_steps=1e4)

In [ ]:
expID = exp_train(specs, deep_eq_model_fn, prefix="ensK-$(now())", train_steps=1e4, merge=true)

In [ ]:
# or I could train with K specifically designed for Sachs-2005


In [ ]:
# construct many dses
ds, test_ds = spec2ds(specs)

In [ ]:
# merge data
function merge_dses(dses)
    # merge datasets
    # 1. merge raw_x and raw_y
    raw_x = cat([ds.raw_x for ds in dses]..., dims=3)
    raw_y = cat([ds.raw_y for ds in dses]..., dims=3)
    # 2. assert batch_size
    batch_size = dses[1].batch_size
    # 4. construct new ds
    DataSetIterator(raw_x, raw_y, batch_size)
end

In [ ]:
size(ds[1].raw_x)

In [ ]:
ds

In [ ]:
merge_dses(ds)

# Some rather random testing

In [ ]:
exp_train(DataSpec(d=10, k=1, gtype=:SF,
        noise=:Gaussian, mechanism=:Linear, mat=:COV),
    deep_eq_model_fn, prefix="test-EQ", train_steps=1e4)

In [ ]:
exp_train(DataSpec(d=10, k=1, gtype=:ER,
        noise=:Gaussian, mechanism=:Linear, mat=:COV),
    deep_eq_model_fn, prefix="test-EQ", train_steps=1e4)

In [ ]:
exp_train(DataSpec(d=20, k=1, gtype=:SF,
        noise=:Gaussian, mechanism=:Linear, mat=:COV),
    deep_eq_model_fn, prefix="test-EQ", train_steps=1e4)

In [ ]:
# Test the model
spec = DataSpec(d=10, k=1, gtype=:SF,
        noise=:Gaussian, mechanism=:Linear, mat=:COV)
expID = "EQ-$(dataspec_to_id(spec))"

In [ ]:
exp_test(expID, spec, use_raw=true)

In [ ]:
spec

In [ ]:
function adhoc_test(expID, spec)
    model, _ = joinpath("saved_models", expID) |> load_most_recent
    model = gpu(model)
    ds, test_ds = spec2ds(spec)
    metrics, t = sup_test_raw(model, test_ds, nbatch=16)
    return metrics
end

In [ ]:
adhoc_test(expID, spec)

In [ ]:
dataspec_to_id(spec)

In [ ]:
_results[expID=>"raw-$(dataspec_to_id(spec))"]